In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request

import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from boto import get_google_secret

import pandas as pd
from square.client import Client

In [2]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

In [3]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

In [4]:

creds = ServiceAccountCredentials.from_json_keyfile_name("googlecreds.json", scopes)


In [5]:
client = gspread.authorize(creds)

In [6]:
sheet = client.open_by_key("1h5hBte_aDD9FwTDoTN1ymSh-LNIBe2wyq1QrQYhSwNE")


In [37]:
match_worksheet = sheet.worksheet('Match Subs')


# Print the DataFrame to verify

# Define the range of data you want (e.g., "A1:D10")
cell_range = match_worksheet.get('A1:I')

# Convert the cell range to a DataFrame
df = pd.DataFrame(cell_range[1:], columns=cell_range[0])  # First row as header




In [38]:
train_worksheet = sheet.worksheet('Training Subs')



# Print the DataFrame to verify

# Define the range of data you want (e.g., "A1:D10")
cell_range = train_worksheet.get('A1:F')

# Convert the cell range to a DataFrame
df_train = pd.DataFrame(cell_range[1:], columns=cell_range[0])  # First row as header




In [9]:
df_train

,Date,Time,Gross Sales,Customer Name,Gross Sales,Transaction ID
0,2024-07-07,10:57:41,5,Paul Ball,5.00,None
1,2024-07-06,11:10:55,5,Martin Goodson,5.00,None
2,2024-07-06,11:10:40,5,Jason Heath,5.00,None
3,2024-07-06,11:10:21,5,Joel Wright,5.00,None
4,2024-07-06,11:09:08,5,Anthony Ball,5.00,None
...,...,...,...,...,...,...
718,2024-09-27,11:19,,Ismail Wall,5.00,NnoqJmqqaCBNK7hz38w2lp5cw0UZY
719,2024-09-27,11:36,,Josh Billings,20.00,V1DxRZFBOjxauMXR6TXDA4rCcXQZY
720,2024-09-27,12:42,,Dom Robbins,5.00,vM22yM7JWDM16D2Othr0NTwgTWaZY
721,2024-09-27,,,Matt Crowe,5.00,None


In [10]:
square_subs = df[df['Channel']=='Square'].copy()
square_subs['Date']=pd.to_datetime(square_subs['Date'])


train_subs = df_train[df_train['Time']!='']
train_subs['Date']=pd.to_datetime(train_subs['Date']).copy()
train_subs

/var/folders/06/z1c103xn2zv4cd1cb30htxdh0000gn/T/ipykernel_86419/806143360.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_subs['Date']=pd.to_datetime(train_subs['Date']).copy()


,Date,Time,Gross Sales,Customer Name,Gross Sales,Transaction ID
0,2024-07-07,10:57:41,5,Paul Ball,5.00,None
1,2024-07-06,11:10:55,5,Martin Goodson,5.00,None
2,2024-07-06,11:10:40,5,Jason Heath,5.00,None
3,2024-07-06,11:10:21,5,Joel Wright,5.00,None
4,2024-07-06,11:09:08,5,Anthony Ball,5.00,None
...,...,...,...,...,...,...
714,2024-09-27,10:24,,Masood Hamidi,5.00,jOyuIwLm547HkWnkHRIYbbdON3ZZY
717,2024-09-27,11:18,,Ismail Wall,5.00,Bl6S3vDPCi3Z1eDAHYFIyMnnFzRZY
718,2024-09-27,11:19,,Ismail Wall,5.00,NnoqJmqqaCBNK7hz38w2lp5cw0UZY
719,2024-09-27,11:36,,Josh Billings,20.00,V1DxRZFBOjxauMXR6TXDA4rCcXQZY


In [11]:
maxd=min(square_subs['Date'].max(), train_subs['Date'].max())
maxds=maxd.strftime("%Y-%m-%dT00:00:00.000Z")
maxds

'2024-09-27T00:00:00.000Z'

In [12]:
client = Client(
    access_token= 'EAAAFO_dqOOnvRZCLDvOtKfBGznlxPy5zb4anviDPIJBjXjXGkmo5d6w1jxoxtCn',#get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [13]:
orders_api = client.orders
customers_api = client.customers

In [14]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
    'query': {
        'filter': {
            
            'date_time_filter': {
                'created_at': {
                    'start_at': f'{maxds}',
                    "end_at": "2025-01-21T15:12:20.000Z",
                }
            },
      # "state_filter": {
        # "states": [
        #   "COMPLETED"
        # ]
      # }
       },
    "sort": {
      "sort_field": "UPDATED_AT",
      "sort_order": "DESC"
    }
        
    },
   }

In [15]:
result = orders_api.search_orders(body)
s_orders = result.body['orders']


i=1
while result.cursor:
    print(i)
    i+=1
    result = orders_api.search_orders(body,
        cursor = result.cursor
    )
    s_orders.append(result.body['orders'])

    # customers.update(customers2)

In [16]:
for x in s_orders:
    try:
        a = x['line_items']
    except:
        print(x)

In [17]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
    print(result.cursor)
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFPMUxuWXg2Njk4eUZoVnBud2d1TkpFSU1Q
2
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFMQ3ZwY1AtMWd3bHNEdUluOVJwR0VFSU1Q
3
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEI2MElQdXpucWJDeGhWcGthamdTeU1FSU1Q
4
CgUIgwcYAQonCIMPEiJpbnY6MC1DaENtdERkNHVFbDk4TzE5MkNCRE81R0dFSU1Q
5
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEExbkQ1T2pGand2U2tydnlHWkxGSzFFSU1Q
6
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEQtbC1Zc0VrakF5LTJmaE1HODVpeU5FSU1Q
7
CgUIgwcYAQonCIMPEiJpbnY6MC1DaENqYmt1dG93bWhuWE1ZMmg2eEdjdFFFSU1Q
8
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFEX1NzaVc2VTBFSE82TDJ5NW5WMldFSU1Q
9
CgUIgwcYAQonCIMPEiJpbnY6MC1DaENfaldHN3VUcUtPUkdicG45VVNMb2ZFSU1Q
10
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFLTzBrd25lYU5KR2JXWGhzbk9uRk9FSU1Q
11
CgUIgwcYAQonCIMPEiJpbnY6MC1DaENKUUVJVU1WWEVPOEdiUEVWRGl6N1lFSU1Q
12
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERUYUdtYnh0OWZPS2IydWcxNHd0dmFFSU1Q
13
CgUIgwcYAQonCIMPEiJpbnY6MC1DaER3SFEyTDlqQURCSUFDYjg1RHItT2dFSU1Q
14
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFHYk5MYmg4Y01uNjBpMDBJdHpxUTZFSU1Q
15
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEFsLTZaZTF6Z3Y3OXBPdlVBdk

,id,version,location_id,order_id,payment_requests,primary_recipient,invoice_number,title,description,public_url,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,sale_or_service_date,store_payment_method_enabled,next_payment_amount_money
0,inv:0-ChDKIEWeQKeF4cucinsiDyC8EIMP,3,SBBJSGR24YHYB,zcLzB6ncIBfHIXzUCjiiAhMAVpTZY,[{'uid': 'd7a75ed3-396b-4de6-9a66-1833666eaaef...,"{'customer_id': '1CSECS76C7VGAHQQ84THPSAGTC', ...",2024-09-24_zcLzB6ncIBfHIXzUCjiiAhMAVpTZY_1,Total Outstanding Training Subs,Total Outstanding Training Subs,https://app.squareup.com/pay-invoice/inv:0-ChD...,PAID,Europe/London,2024-09-24T09:39:45Z,2024-09-27T07:38:44Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2024-09-24,False,NaN
1,inv:0-ChCVk93Fvq_YcmHx7bvfMDnKEIMP,2,SBBJSGR24YHYB,baiUTj6BvKO6jg7Kt7KbfcUB3gHZY,[{'uid': 'd4942672-8052-48d6-97e9-99ac2d48b791...,"{'customer_id': 'VV07SF0MN54AVE0ZRWNQYBAF5W', ...",2024-09-24_baiUTj6BvKO6jg7Kt7KbfcUB3gHZY_1,Total Outstanding Training Subs,Total Outstanding Training Subs,https://app.squareup.com/pay-invoice/inv:0-ChC...,UNPAID,Europe/London,2024-09-24T09:39:44Z,2024-09-24T11:00:02Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2024-09-24,False,"{'amount': 1000, 'currency': 'GBP'}"
2,inv:0-ChCtRUc0-xc86yi19WolernuEIMP,3,SBBJSGR24YHYB,dnAgliohM5yUiYyOgjbDj2UODPHZY,[{'uid': '2cfff9bc-6964-4544-acc5-1f0344ad65a3...,"{'customer_id': 'T1QTC2PQJGS9HFH4ECEFYB8DYW', ...",2024-09-24_dnAgliohM5yUiYyOgjbDj2UODPHZY_1,Total Outstanding Training Subs,Total Outstanding Training Subs,https://app.squareup.com/pay-invoice/inv:0-ChC...,PAID,Europe/London,2024-09-24T09:39:43Z,2024-09-24T11:02:22Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2024-09-24,False,NaN
3,inv:0-ChBAb1cjInRpQmfqD7RBa6X3EIMP,2,SBBJSGR24YHYB,pLzMsxNs4i7SzjSMCUgFSbNgthBZY,[{'uid': '68a61e4e-3d74-4b57-b1e9-46be4bc0061a...,"{'customer_id': '8DDMF6VXNV5J9WT2R9SBFF0MKG', ...",2024-09-24_pLzMsxNs4i7SzjSMCUgFSbNgthBZY_1,Total Outstanding Training Subs,Total Outstanding Training Subs,https://app.squareup.com/pay-invoice/inv:0-ChB...,UNPAID,Europe/London,2024-09-24T09:39:42Z,2024-09-24T11:00:02Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2024-09-24,False,"{'amount': 500, 'currency': 'GBP'}"
4,inv:0-ChCQNn8rfL7tg4qyImaTxVyCEIMP,3,SBBJSGR24YHYB,tZhmtMrjvV7nl26PnYc65BL5abIZY,[{'uid': 'ab5caac3-88f3-42af-aee3-66f434d545a1...,"{'customer_id': 'YVZKQG6MSWVQ6JG5JSAREAM56R', ...",2024-09-24_tZhmtMrjvV7nl26PnYc65BL5abIZY_1,Total Outstanding Training Subs,Total Outstanding Training Subs,https://app.squareup.com/pay-invoice/inv:0-ChC...,PAID,Europe/London,2024-09-24T09:39:41Z,2024-09-24T11:45:56Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2024-09-24,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,inv:0-ChDPM4kl4NAiP5pXozC1Y_ESEIMP,2,SBBJSGR24YHYB,hpQSgBsF0IoWkRnzqfCzbvYClWZZY,[{'uid': '35e32c13-7cdc-449f-b072-478fd634b433...,"{'customer_id': 'GB6X1PRCQZHR7BW76TTDCADACW', ...",000052,,,https://app.squareup.com/pay-invoice/inv:0-ChD...,UNPAID,Europe/London,2023-06-29T11:10:01Z,2024-05-28T10:03:08Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-03-30,False,"{'amount': 500, 'currency': 'GBP'}"
18,inv:0-ChAUAlcEksn2SoFvemS9qv1AEIMP,1,SBBJSGR24YHYB,NrGYUEcjQYCbGN78OXCQDLkMrfYZY,[{'uid': '1bfd6dd0-1452-4d9e-905c-556bc79f0ba8...,"{'customer_id': 'DVXQMTVG4MFTHQ765VFCSDCZC8', ...",000053,,,https://app.squareup.com/pay-invoice/inv:0-ChA...,UNPAID,Europe/London,2023-06-29T11:44:40Z,2024-05-28T10:03:13Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,"{'amount': 500, 'currency': 'GBP'}"
19,inv:0-ChBb4hlJg_JLA3LSygeItKZ3EIMP,2,SBBJSGR24YHYB,BjVuDsGV2pM5uLwRafpuX2asMIEZY,[{'uid': '20c3dd06-10de-444b-82e3-0b20104b9048...,"{'customer_id': 'X2KSKJ4GYN0M7G01D6SYKT1BHC', ...",000048,,,https://app.squareup.com/pay-invoice/inv:0-ChB...,UNPAID,Europe/London,2023-06-29T10:59:36Z,2024-05-28T10:03:06Z,"{'card': True, 'square_gift_ca

In [18]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

In [19]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'updated_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    new_order['tenders']=get_value(order,'tenders')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        pass
        invoice = invoices[invoices['order_id']==new_order['id']]
        
        
        if len(invoice)>0:
            cust_id = list(invoice['primary_recipient'])[0]['customer_id']
            customer = customers_api.retrieve_customer(cust_id)
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)

    if 'line_items' in order: 
        for item in order['line_items']:
            line_dict = new_order.copy()
            
            line_dict['variation_name']=get_value(item,'variation_name')
            line_dict['item_name']=get_value(item,'name')

            # if 'raining' in line_dict['item_name']:
            #     print(item)
            line_dict['note']=get_value(item,'note')
            line_dict['total_money']=item['total_money']['amount']/100
            orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 211
1 / 211
2 / 211
3 / 211
4 / 211
5 / 211
6 / 211
7 / 211
8 / 211
9 / 211
10 / 211
11 / 211
12 / 211
13 / 211
14 / 211
15 / 211
16 / 211
17 / 211
18 / 211
19 / 211
20 / 211
21 / 211
22 / 211
23 / 211
24 / 211
25 / 211
26 / 211
27 / 211
28 / 211
29 / 211
30 / 211
31 / 211
32 / 211
33 / 211
34 / 211
35 / 211
36 / 211
37 / 211
38 / 211
39 / 211
40 / 211
41 / 211
42 / 211
43 / 211
44 / 211
45 / 211
46 / 211
47 / 211
48 / 211
49 / 211
50 / 211
51 / 211
52 / 211
53 / 211
54 / 211
55 / 211
56 / 211
57 / 211
58 / 211
59 / 211
60 / 211
61 / 211
62 / 211
63 / 211
64 / 211
65 / 211
66 / 211
67 / 211
68 / 211
69 / 211
70 / 211
71 / 211
72 / 211
73 / 211
74 / 211
75 / 211
76 / 211
77 / 211
78 / 211
79 / 211
80 / 211
81 / 211
82 / 211
83 / 211
84 / 211
85 / 211
86 / 211
87 / 211
88 / 211
89 / 211
90 / 211
91 / 211
92 / 211
93 / 211
94 / 211
95 / 211
96 / 211
97 / 211
98 / 211
99 / 211
100 / 211
101 / 211
102 / 211
103 / 211
104 / 211
105 / 211
106 / 211
107 / 211
108 / 211
109 / 211
110 / 211


In [20]:
orders_df[['variation_name', 'item_name']].drop_duplicates()

,variation_name,item_name
0,None,u23 Match Subs
2,Regular,Crisps
3,Regular,Lucozade
4,Regular,Coffee
7,Regular,Chocolate
9,Regular,Tea
18,Regular,Diet Coke
19,Regular,Water
26,Regular,Red Bull
33,Regular,Nuts


In [21]:
match_square = orders_df[orders_df['item_name'].str.lower().str.contains('match')]
match_square=match_square[match_square['first_name'].notnull()]
match_square['Transaction ID']=match_square['id']
match_square

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money,Transaction ID
0,fAi37gGYVHb8SOdJAHU5CxdtE2NZY,2024-09-29T18:03:28.000Z,CCWQB3GAF5R7A98CYZBYEAEBA8,OPEN,"[{'id': 'NcRPOMcMNizwzUQC8TrNZ4k0H16YY', 'loca...",Magnus,Connolly,None,u23 Match Subs,None,5.0,fAi37gGYVHb8SOdJAHU5CxdtE2NZY
1,TaIOle43BMY4JWMYastN05vuOMEZY,2024-09-29T14:02:21.000Z,436J8X0SV9GMCJKY5N79CZ41QR,OPEN,"[{'id': 'n9Jc35wT4eEJuEhbVxK6LjlsZ2YZY', 'loca...",Oliver,Lea,None,u23 Match Subs,None,5.0,TaIOle43BMY4JWMYastN05vuOMEZY
56,xByRDIdU8yhA67jYEmMPhwMN5eTZY,2024-09-28T21:24:49.000Z,T804334RB9541D6WT8Q9J9HKR8,OPEN,"[{'id': 'L94M1nBuvJP9tJhcJrVBbriabv5YY', 'loca...",Sean,Rowlett,None,u23 Match Subs,None,5.0,xByRDIdU8yhA67jYEmMPhwMN5eTZY
57,7tykcgQtzlcxZzdudd2q6RweV,2024-09-28T18:59:58.000Z,HT08JK41Q90B3AT9XBHM539HZC,COMPLETED,"[{'id': 'TzlGBBPVNgd9dCOCdKgVSsOFvaB', 'locati...",James,Rodriguez,Full Game,Match Sub,None,10.0,7tykcgQtzlcxZzdudd2q6RweV
61,7eMWIfSSVwxb9DeSo12PxsqwoCMZY,2024-09-28T18:25:07.000Z,M1SEP4EB9S4T39ANX9MBY5ZAVW,OPEN,"[{'id': 'dCExwJQAMkZ30MWEaHdSVBkkjYEZY', 'loca...",Masood,Hamidi,None,Match Subs,None,10.0,7eMWIfSSVwxb9DeSo12PxsqwoCMZY
70,rCWKPWjPsp8wSPvqPdgaOYU5GHFZY,2024-09-28T17:44:22.000Z,2Q3DKK7ER44TZS75Q93KXHT71W,OPEN,"[{'id': 'Th82JJVmZI5yethST5LwmjON1HbZY', 'loca...",George,Havord,None,Match Subs,None,10.0,rCWKPWjPsp8wSPvqPdgaOYU5GHFZY
100,5IWHOVausx3TXcxKAvq2oC3eV,2024-09-28T16:21:24.000Z,65CKDVXENCTYXBGMTVXAK1ZQ8G,COMPLETED,"[{'id': '9KAZWCgkOt2GMP3keZcwjvtdvaB', 'locati...",Tam de,Lacey,Full Game,Match Sub,None,10.0,5IWHOVausx3TXcxKAvq2oC3eV
112,1uMyh6fTGzrPoGBUJxOYr8heV,2024-09-28T16:01:55.000Z,7ZFNY7NNCD29K1WR0CBBQ4F2RR,COMPLETED,"[{'id': '7jppFPMBBgqeAVZ4WqJXmBQqvaB', 'locati...",Josh,Sherriff-Jolly,Full Game,Match Sub,None,10.0,1uMyh6fTGzrPoGBUJxOYr8heV
266,FsvS53c1ozc4D9HA4gMrlAweV,2024-09-28T13:57:24.000Z,X4KWYY5JN97Q9CDANDJJP3D4GC,COMPLETED,"[{'id': '5IIdMLZunJMjgvyTa2PBSz5JuaB', 'locati...",Connor,White,Half Game,Match Sub,None,5.0,FsvS53c1ozc4D9HA4gMrlAweV
284,DHJSQYNH3RYsXvDqy0IJ7mteV,2024-09-28T13:48:10.000Z,PJ288BDWZS5ANCARFGC3VXWVDM,COMPLETED,"[{'id': 'LXh8oeoOV1LIcVQl1pcR6hJnuaB', 'locati...",Dan,Dan,U23 Full Game,Match Sub,None,5.0,DHJSQYNH3RYsXvDqy0IJ7mteV


In [22]:
train_square = orders_df[orders_df['item_name'].str.lower().str.contains('raining')]
train_square=train_square[train_square['first_name'].notnull()]
train_square['Transaction ID']=train_square['id']
train_square

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money,Transaction ID
39,jUxSt1LYFsvgX1T8MJuiY00neGQZY,2024-09-29T08:46:57.000Z,B735XZ08PZX5T8ZMVCEDWH2R6C,OPEN,"[{'id': 'rLlB4nsW8CVA79J5no9J6C8tvUGZY', 'loca...",Ciaran,Curran,None,Training,None,5.0,jUxSt1LYFsvgX1T8MJuiY00neGQZY
268,FsvS53c1ozc4D9HA4gMrlAweV,2024-09-28T13:57:24.000Z,X4KWYY5JN97Q9CDANDJJP3D4GC,COMPLETED,"[{'id': '5IIdMLZunJMjgvyTa2PBSz5JuaB', 'locati...",Connor,White,Regular,Training Sub,None,5.0,FsvS53c1ozc4D9HA4gMrlAweV
307,vM22yM7JWDM16D2Othr0NTwgTWaZY,2024-09-27T11:42:59.000Z,NE45HCW3B972V8P75444MDGB2G,OPEN,"[{'id': '5mtMIILMj8SBfFpEhUHGubVUeNJZY', 'loca...",Dom,Robbins,None,Training,None,5.0,vM22yM7JWDM16D2Othr0NTwgTWaZY
309,NnoqJmqqaCBNK7hz38w2lp5cw0UZY,2024-09-27T10:19:08.000Z,52039G36JS0ED9ETH55X2748PC,OPEN,"[{'id': 'vFTCc3EbxAk7aI8uXUlr8SYdZD7YY', 'loca...",Ismail,Wall,None,Training,None,5.0,NnoqJmqqaCBNK7hz38w2lp5cw0UZY
310,Bl6S3vDPCi3Z1eDAHYFIyMnnFzRZY,2024-09-27T10:18:47.000Z,52039G36JS0ED9ETH55X2748PC,OPEN,"[{'id': 'DTxAkd6XGqnWESSh3HuR01SSFb7YY', 'loca...",Ismail,Wall,None,Training,None,5.0,Bl6S3vDPCi3Z1eDAHYFIyMnnFzRZY
312,jOyuIwLm547HkWnkHRIYbbdON3ZZY,2024-09-27T09:24:05.000Z,M1SEP4EB9S4T39ANX9MBY5ZAVW,OPEN,"[{'id': 'huhjtexSPYX7HVRBuXQA5aESRSAZY', 'loca...",Masood,Hamidi,None,Training,None,5.0,jOyuIwLm547HkWnkHRIYbbdON3ZZY
315,nORSozFhz3BtPiKYiBZpDtqS1kaZY,2024-09-27T08:41:52.000Z,N08M19NRFGZTV3XTYJ2F7G7A7C,OPEN,"[{'id': 'li0cJhfh8sFTbe7uP2MlfGP9gcXZY', 'loca...",William,Sewell,None,Training,None,5.0,nORSozFhz3BtPiKYiBZpDtqS1kaZY


In [23]:
trans = df[['Transaction ID']].drop_duplicates()
trans=trans[trans['Transaction ID']!='']
trans['FLAG']=1
trans

,Transaction ID,FLAG
1,3OdieYfzThSs0sk9wXsUOXfzL3LZY,1
2,tWE3lnpLqQTOv0UtSTgcXj3eV,1
3,9cfn6LknlYXeal75j01uaooeV,1
4,vRpBIpDIeg85drfTgh5qly0eV,1
5,vKvbQvGI9MfxiOy9uZm9ZWdNcLIZY,1
...,...,...
467,tFhThMvdDc0J9m3flkjwrQMF4laZY,1
468,7mCxVdQBAquB9ZCMUILqeER50tcZY,1
469,dlfCJW9TCGonMzzuv3IHltpUlGTZY,1
470,LKOcdq1HilVbJpXF9YuyAmaqF6YZY,1


In [24]:
match_mg = pd.merge(left=match_square,right=trans, how='left', on=['Transaction ID'])
match_to_upload=match_mg[match_mg['FLAG']!=1].copy()
match_to_upload

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money,Transaction ID,FLAG
0,fAi37gGYVHb8SOdJAHU5CxdtE2NZY,2024-09-29T18:03:28.000Z,CCWQB3GAF5R7A98CYZBYEAEBA8,OPEN,"[{'id': 'NcRPOMcMNizwzUQC8TrNZ4k0H16YY', 'loca...",Magnus,Connolly,None,u23 Match Subs,None,5.0,fAi37gGYVHb8SOdJAHU5CxdtE2NZY,NaN
1,TaIOle43BMY4JWMYastN05vuOMEZY,2024-09-29T14:02:21.000Z,436J8X0SV9GMCJKY5N79CZ41QR,OPEN,"[{'id': 'n9Jc35wT4eEJuEhbVxK6LjlsZ2YZY', 'loca...",Oliver,Lea,None,u23 Match Subs,None,5.0,TaIOle43BMY4JWMYastN05vuOMEZY,NaN
2,xByRDIdU8yhA67jYEmMPhwMN5eTZY,2024-09-28T21:24:49.000Z,T804334RB9541D6WT8Q9J9HKR8,OPEN,"[{'id': 'L94M1nBuvJP9tJhcJrVBbriabv5YY', 'loca...",Sean,Rowlett,None,u23 Match Subs,None,5.0,xByRDIdU8yhA67jYEmMPhwMN5eTZY,NaN
3,7tykcgQtzlcxZzdudd2q6RweV,2024-09-28T18:59:58.000Z,HT08JK41Q90B3AT9XBHM539HZC,COMPLETED,"[{'id': 'TzlGBBPVNgd9dCOCdKgVSsOFvaB', 'locati...",James,Rodriguez,Full Game,Match Sub,None,10.0,7tykcgQtzlcxZzdudd2q6RweV,NaN
4,7eMWIfSSVwxb9DeSo12PxsqwoCMZY,2024-09-28T18:25:07.000Z,M1SEP4EB9S4T39ANX9MBY5ZAVW,OPEN,"[{'id': 'dCExwJQAMkZ30MWEaHdSVBkkjYEZY', 'loca...",Masood,Hamidi,None,Match Subs,None,10.0,7eMWIfSSVwxb9DeSo12PxsqwoCMZY,NaN
5,rCWKPWjPsp8wSPvqPdgaOYU5GHFZY,2024-09-28T17:44:22.000Z,2Q3DKK7ER44TZS75Q93KXHT71W,OPEN,"[{'id': 'Th82JJVmZI5yethST5LwmjON1HbZY', 'loca...",George,Havord,None,Match Subs,None,10.0,rCWKPWjPsp8wSPvqPdgaOYU5GHFZY,NaN
6,5IWHOVausx3TXcxKAvq2oC3eV,2024-09-28T16:21:24.000Z,65CKDVXENCTYXBGMTVXAK1ZQ8G,COMPLETED,"[{'id': '9KAZWCgkOt2GMP3keZcwjvtdvaB', 'locati...",Tam de,Lacey,Full Game,Match Sub,None,10.0,5IWHOVausx3TXcxKAvq2oC3eV,NaN
7,1uMyh6fTGzrPoGBUJxOYr8heV,2024-09-28T16:01:55.000Z,7ZFNY7NNCD29K1WR0CBBQ4F2RR,COMPLETED,"[{'id': '7jppFPMBBgqeAVZ4WqJXmBQqvaB', 'locati...",Josh,Sherriff-Jolly,Full Game,Match Sub,None,10.0,1uMyh6fTGzrPoGBUJxOYr8heV,NaN
8,FsvS53c1ozc4D9HA4gMrlAweV,2024-09-28T13:57:24.000Z,X4KWYY5JN97Q9CDANDJJP3D4GC,COMPLETED,"[{'id': '5IIdMLZunJMjgvyTa2PBSz5JuaB', 'locati...",Connor,White,Half Game,Match Sub,None,5.0,FsvS53c1ozc4D9HA4gMrlAweV,NaN
9,DHJSQYNH3RYsXvDqy0IJ7mteV,2024-09-28T13:48:10.000Z,PJ288BDWZS5ANCARFGC3VXWVDM,COMPLETED,"[{'id': 'LXh8oeoOV1LIcVQl1pcR6hJnuaB', 'locati...",Dan,Dan,U23 Full Game,Match Sub,None,5.0,DHJSQYNH3RYsXvDqy0IJ7mteV,NaN


In [25]:
df_train

,Date,Time,Gross Sales,Customer Name,Gross Sales,Transaction ID
0,2024-07-07,10:57:41,5,Paul Ball,5.00,None
1,2024-07-06,11:10:55,5,Martin Goodson,5.00,None
2,2024-07-06,11:10:40,5,Jason Heath,5.00,None
3,2024-07-06,11:10:21,5,Joel Wright,5.00,None
4,2024-07-06,11:09:08,5,Anthony Ball,5.00,None
...,...,...,...,...,...,...
718,2024-09-27,11:19,,Ismail Wall,5.00,NnoqJmqqaCBNK7hz38w2lp5cw0UZY
719,2024-09-27,11:36,,Josh Billings,20.00,V1DxRZFBOjxauMXR6TXDA4rCcXQZY
720,2024-09-27,12:42,,Dom Robbins,5.00,vM22yM7JWDM16D2Othr0NTwgTWaZY
721,2024-09-27,,,Matt Crowe,5.00,None


In [26]:
trans = df_train[['Transaction ID']].drop_duplicates()
trans=trans[trans['Transaction ID']!='']
trans['FLAG']=1
trans

,Transaction ID,FLAG
0,None,1
712,zcLzB6ncIBfHIXzUCjiiAhMAVpTZY,1
713,nORSozFhz3BtPiKYiBZpDtqS1kaZY,1
714,jOyuIwLm547HkWnkHRIYbbdON3ZZY,1
717,Bl6S3vDPCi3Z1eDAHYFIyMnnFzRZY,1
718,NnoqJmqqaCBNK7hz38w2lp5cw0UZY,1
719,V1DxRZFBOjxauMXR6TXDA4rCcXQZY,1
720,vM22yM7JWDM16D2Othr0NTwgTWaZY,1


In [54]:
train_mg = pd.merge(left=train_square,right=trans, how='left', on=['Transaction ID'])
train_to_upload=train_mg[train_mg['FLAG']!=1].copy()
train_to_upload

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money,Transaction ID,FLAG
0,jUxSt1LYFsvgX1T8MJuiY00neGQZY,2024-09-29T08:46:57.000Z,B735XZ08PZX5T8ZMVCEDWH2R6C,OPEN,"[{'id': 'rLlB4nsW8CVA79J5no9J6C8tvUGZY', 'loca...",Ciaran,Curran,None,Training,None,5.0,jUxSt1LYFsvgX1T8MJuiY00neGQZY,NaN
1,FsvS53c1ozc4D9HA4gMrlAweV,2024-09-28T13:57:24.000Z,X4KWYY5JN97Q9CDANDJJP3D4GC,COMPLETED,"[{'id': '5IIdMLZunJMjgvyTa2PBSz5JuaB', 'locati...",Connor,White,Regular,Training Sub,None,5.0,FsvS53c1ozc4D9HA4gMrlAweV,NaN


In [55]:
match_to_upload['Date']=match_to_upload['created_at'].apply(lambda x: x[:10])
match_to_upload['Time']=match_to_upload['created_at'].apply(lambda x: x[11:19])
match_to_upload['Net Sales']=match_to_upload['total_money']
match_to_upload['Customer Name']=match_to_upload['first_name'] + ' ' + match_to_upload['last_name']
match_to_upload['Description']='Match Sub (Full Game) - TEst'
match_to_upload['Payment ID']=''
match_to_upload['Channel']='Square'
match_to_upload_final = match_to_upload[['Date', 'Time', 'Net Sales', 'Transaction ID', 'Payment ID', 'Customer Name', 'Description', 'Channel']]
match_to_upload_final
                                                            
                                                        
                                            

,Date,Time,Net Sales,Transaction ID,Payment ID,Customer Name,Description,Channel
0,2024-09-29,18:03:28,5.0,fAi37gGYVHb8SOdJAHU5CxdtE2NZY,,Magnus Connolly,Match Sub (Full Game) - TEst,Square
1,2024-09-29,14:02:21,5.0,TaIOle43BMY4JWMYastN05vuOMEZY,,Oliver Lea,Match Sub (Full Game) - TEst,Square
2,2024-09-28,21:24:49,5.0,xByRDIdU8yhA67jYEmMPhwMN5eTZY,,Sean Rowlett,Match Sub (Full Game) - TEst,Square
3,2024-09-28,18:59:58,10.0,7tykcgQtzlcxZzdudd2q6RweV,,James Rodriguez,Match Sub (Full Game) - TEst,Square
4,2024-09-28,18:25:07,10.0,7eMWIfSSVwxb9DeSo12PxsqwoCMZY,,Masood Hamidi,Match Sub (Full Game) - TEst,Square
5,2024-09-28,17:44:22,10.0,rCWKPWjPsp8wSPvqPdgaOYU5GHFZY,,George Havord,Match Sub (Full Game) - TEst,Square
6,2024-09-28,16:21:24,10.0,5IWHOVausx3TXcxKAvq2oC3eV,,Tam de Lacey,Match Sub (Full Game) - TEst,Square
7,2024-09-28,16:01:55,10.0,1uMyh6fTGzrPoGBUJxOYr8heV,,Josh Sherriff-Jolly,Match Sub (Full Game) - TEst,Square
8,2024-09-28,13:57:24,5.0,FsvS53c1ozc4D9HA4gMrlAweV,,Connor White,Match Sub (Full Game) - TEst,Square
9,2024-09-28,13:48:10,5.0,DHJSQYNH3RYsXvDqy0IJ7mteV,,Dan Dan,Match Sub (Full Game) - TEst,Square


In [56]:
train_to_upload['Date']=train_to_upload['created_at'].apply(lambda x: x[:10])
train_to_upload['Time']=train_to_upload['created_at'].apply(lambda x: x[11:19])
train_to_upload['Gross Sales']=train_to_upload['total_money']
train_to_upload['Customer Name']=train_to_upload['first_name'] + ' ' + train_to_upload['last_name']
# match_to_upload['Description']='Match Sub (Full Game) - TEst'
# match_to_upload['Payment ID']=''
# match_to_upload['Channel']='Square'
train_to_upload_final = train_to_upload[['Date', 'Time', 'Gross Sales', 'Customer Name', 'Gross Sales', 'Transaction ID']]
                                                            
                                                        
train_to_upload_final

,Date,Time,Gross Sales,Customer Name,Gross Sales,Transaction ID
0,2024-09-29,08:46:57,5.0,Ciaran Curran,5.0,jUxSt1LYFsvgX1T8MJuiY00neGQZY
1,2024-09-28,13:57:24,5.0,Connor White,5.0,FsvS53c1ozc4D9HA4gMrlAweV


In [61]:
import datetime as dt
def date_to_serial(date):
    epoch = dt.datetime(1899, 12, 30)
    return (date - epoch).days + (date - epoch).seconds / 86400  # Include fraction for time if necessary

# Apply the date serial conversion for the 'Join Date' column
train_to_upload_final['Date']=pd.to_datetime(train_to_upload_final['Date'])
train_to_upload_final['Date'] = train_to_upload_final['Date'].apply(date_to_serial)
match_to_upload_final['Date']=pd.to_datetime(match_to_upload_final['Date'])
match_to_upload_final['Date'] = match_to_upload_final['Date'].apply(date_to_serial)

/var/folders/06/z1c103xn2zv4cd1cb30htxdh0000gn/T/ipykernel_86419/237503448.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_to_upload_final['Date']=pd.to_datetime(train_to_upload_final['Date'])
/var/folders/06/z1c103xn2zv4cd1cb30htxdh0000gn/T/ipykernel_86419/237503448.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_to_upload_final['Date'] = train_to_upload_final['Date'].apply(date_to_serial)
/var/folders/06/z1c103xn2zv4cd1cb30htxdh0000gn/T/ipykernel_86419/237503448.py:9: SettingWithCopyWa

In [63]:
data_list = match_to_upload_final.values.tolist()
data_list



last_row = len(match_worksheet.get_all_values()) 
next_row = last_row + 1  # Start appending after the last row

# # Prepare the range where you want to update (adjust the column range as needed)
cell_range = f'A{next_row}:I{next_row + len(data_list) - 1}'

# # Update the range with the data
match_worksheet.update(cell_range, data_list, value_input_option='RAW')

{'spreadsheetId': '1h5hBte_aDD9FwTDoTN1ymSh-LNIBe2wyq1QrQYhSwNE',
 'updatedRange': "'Match Subs'!A493:H506",
 'updatedRows': 14,
 'updatedColumns': 8,
 'updatedCells': 112}

In [60]:
data_list = train_to_upload_final.values.tolist()
data_list



last_row = len(train_worksheet.get_all_values()) 
next_row = last_row + 1  # Start appending after the last row

# # Prepare the range where you want to update (adjust the column range as needed)
cell_range = f'A{next_row}:F{next_row + len(data_list) - 1}'

# # Update the range with the data
train_worksheet.update(cell_range, data_list, value_input_option='RAW')

/Users/james.timmins/Documents/OTs_square/myenv/lib/python3.11/site-packages/gspread/worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1h5hBte_aDD9FwTDoTN1ymSh-LNIBe2wyq1QrQYhSwNE',
 'updatedRange': "'Training Subs'!A725:F726",
 'updatedRows': 2,
 'updatedColumns': 6,
 'updatedCells': 12}